In [1]:
import os
from glob import glob
import threading

import numpy as np
import dask.dataframe as dd
from dask import delayed
import pandas as pd
import xarray as xr

%matplotlib inline
from matplotlib import pyplot as plt
#import matplotlib.animation as anim

import crocosi.postp as pp
from crocosi.jet import set_relevant_time

---

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.scale(28*2)

In [3]:
from dask.distributed import Client
#client = Client()  # set up local cluster on your laptop
client = Client(cluster) # with distributed cluster

In [6]:
client

Client Scheduler: tcp://10.148.1.2:32783 Dashboard: http://10.148.1.2:8787/status,Cluster Workers: 28 Cores: 28 Memory: 120.12 GB


---

In [ ]:
#root_path = '/Users/zcaspar/'
root_path = '/home/datawork-lops-osi/slgentil/croco/jetn/'
run = 'jet_cfg1_wp75_4km_1500a2000j_floats_lev50_itide/'

gparams = {'f0': 1.0313e-4, 'beta': 1.6186e-11}
r = pp.CROCOrun(root_path+run, prefix='file_', open_nc=['surf'], 
                chunk_time={'his': 1, 'surf': 1}, grid_params=gparams)
#set_relevant_time(r) # this triggers slow computations later on, don't know why

In [8]:
#path_data='/Users/zcaspar/jet_cfg1_wp75_4km_1500a2000j_floats_lev50_itide/'
#df = dd.read_csv(glob(root_path+run+'t?/float.????'), 
df = dd.read_csv(glob(root_path+run+'t1/float.????'), 
                 names=['id','time','xgrid','ygrid','zgrid','depth','temp','X1','X2','X3','X4','X5'],
                 delim_whitespace=True).persist()
df.head()

,id,time,xgrid,ygrid,zgrid,depth,temp,X1,X2,X3,X4,X5
0,7161,1500.0069,196.40944,541.79652,50.0,-0.935944,25.886666,0.058079,-0.105506,-5.820925e-08,3.420817e-07,-8.260263
1,7162,1500.0069,196.40937,543.30184,50.0,-0.940022,25.885229,0.057283,-0.101750,-2.034746e-08,3.515808e-07,-8.300321
2,7163,1500.0069,196.40946,544.80716,50.0,-0.944117,25.884374,0.058512,-0.097537,-1.201106e-09,3.599363e-07,-8.340519
3,7164,1500.0069,196.40961,546.31250,50.0,-0.948345,25.884627,0.060480,-0.093268,-1.958836e-08,3.627984e-07,-8.381996
4,7165,1500.0069,196.40971,547.81780,50.0,-0.952706,25.885603,0.061626,-0.089432,-6.096121e-08,3.575484e-07,-8.424740


In [18]:
# sort right away data by time:
#df = df.apply(lambda d: d.sort_values('time')).persist()
# not working, not implemented in dask (too much shufling)

In [19]:
# note for AP for future stuff
#df.groupby(['id']).get_group(2).compute()
#df.groupby(['id']).apply(lambda d: d.sort_values('time'), meta=df).to_delayed()
# with stack to get dask array

### paramètres :

In [9]:
#Dt = 6*6 # Dt = X*6 images toutes les Xh
dt = 2 # temps avant l'instant t plotté utilisé pour visualiser la trajectoire (en jours)
#ti = 1500 # temps0 en jours
#tf = 1599 # temps d'arrêt en jours
vari = 'ssh' # variable affichée en fond pour une seule variable
#vari1 = 'u'  # 1ère variable affichée en fond pour 2 variables
#vari2 = 'v'  # 2ème variable affichée en fond pour 2 variables
#floats_var = 'depth' # variable affichée le long de la trajectoire du drifter
#float_id = [102,300] # numéro des flotteurs voulu

In [10]:
# could delete all figures beforehand
figdir = '/home1/scratch/aponte/figs/'
print('Figures directory: '+figdir)

def genfig(i, ds, v, df, overwrite=True):
    
    figname = figdir+'float_%s_t%05d' %(v, i)+'.png'
    
    if not os.path.isfile(figname) or overwrite:
        time = ds['time_instant'].values
        ldf = df[ (df.time>=time-dt) & (df.time<=time) ]
        #
        MPL_LOCK = threading.Lock()
        with MPL_LOCK:
            plt.switch_backend('agg')    
            #
            fig = plt.figure(figsize=(12,12))
            ax = fig.add_subplot(1,2,1)        
            ds[v].plot(ax=ax)
            ax.plot(ldf.xgrid, ldf.ygrid, '.', color='k')
            fig.savefig(figname)
            #
            m = 1.
    else:
        m = -1.
    return m

Figures directory: /home1/scratch/aponte/figs/


In [11]:
ds = r['surf'][vari].to_dataset()
#v = (r['surf'][vari].shift(time=-1)+r['surf'][vari])*.5
#t = ds.time

In [12]:
ds

<xarray.Dataset>
Dimensions:       (time: 72000, x_rho: 258, y_rho: 722)
Coordinates:
    xi_rho        (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
    eta_rho       (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
    time_instant  (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    time_counter  (time) float64 1.5e+03 1.5e+03 1.5e+03 ... 2e+03 2e+03 2e+03
    f             (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
Dimensions without coordinates: time, x_rho, y_rho
Data variables:
    ssh           (time, y_rho, x_rho) float32 dask.array<chunksize=(1, 722, 258), meta=np.ndarray>

In [13]:
rg = range(0,6*24*3) # 3 days
II = np.array_split(rg,len(rg)/56) # 4 maybe will work
print('%d loops to be done'%len(II))
for I in II:
    print('  %d - %d'%(I[0],I[-1]))
    lt = ds.time_instant.isel(time=I).values
    ldf = df[ (df.time>lt[0]-dt) & (df.time<lt[-1]) ]
    values = [delayed(genfig)(i, ds.isel(time=i), vari, \
                              ldf, \
                              overwrite=True) for i in I]
    futures = client.compute(values)
    results = client.gather(futures)

7 loops to be done
 - 0 - 61
 - 62 - 123
 - 124 - 185
 - 186 - 247
 - 248 - 309
 - 310 - 370
 - 371 - 431


In [ ]:
cluster.close()